# Coordinated Multipoint Using a Quantum Computer
This notebook demonstrates

## What is Coordinated Multipoint?

TBD: add description, papers, etc

# Modeling a Network

TBD: explain the King's graph-based network model

The first code cell imports the needed funcions
NOTE: The code imports from "helpers", a folder colocated with this Jupyter Notebook.

In [ ]:
import matplotlib.pyplot as plt
from dwave.system import DWaveSampler
from helpers.draw import draw_network
from helpers.filters import apply_filter, apply_filters, compare_signals, create_filter, create_filters, time_filter_instantiation
from helpers.network import configure_network, create_channels, print_network_stats, simulate_signals
%matplotlib inline 

## Create a Network Graph

TBD: explain the King's graph-based network model

NOTE: The code imports from "helpers", a folder colocated with this Jupyter Notebook.

In [ ]:
network_size = 4

network, _ = configure_network(lattice_size=network_size)
num_tx, num_rx = print_network_stats(network)

draw_network(network)

The next code cell plots the network graph, showing transmitters in red and receivers in green. 

## Create Channels

Between the transmitters and receivers there are channels etc

In [ ]:
channels = create_channels(network)

print(f"`channels` is a {channels.shape[0]} by {channels.shape[1]} matrix. Its top-left value is {channels[0, 0]}.")

# Decoding CoMP Transmissions

TBD: explain linear filters

## Create Filters
Explain about filters

In [ ]:
filters = create_filters(channels)
print(f"Created filters: {list(filters.keys())}.")

## Simulate Transmitted Signals


In [ ]:
y, transmitted_symbols = simulate_signals(channels)                 
print(y.flatten())

## Decode Received Signals

In [ ]:
v = apply_filters(y, filters)
compare_signals(v, transmitted_symbols)

# Scaling Up

Move code to a helpers/function

In [ ]:
from helpers.filters import time_filter_instantiation

time_filter_instantiation(network_size=[5, 10, 15])

# Solving with a Quantum Computer
Some words here

In [ ]:
qpu = DWaveSampler(solver=dict(topology__type="pegasus"))

print(f"Selected {qpu.solver.name} with {len(qpu.nodelist)} qubits.")

## Representing the Problem as a BQM

### Create a Network

In [ ]:
import dimod
network, embedding = configure_network(lattice_size=16, ratio=1, qpu=qpu)
num_tx, num_rx = print_network_stats(network)

am = dimod.generators.mimo.lattice_to_attenuation_matrix(network)[0]
channels, _ = dimod.generators.mimo.create_channel(num_receivers=num_rx, 
                                                               num_transmitters=num_tx, 
                                                               F_distribution=("binary", "real"),
                                                               attenuation_matrix=am)

### Create Filters and Transmissions

In [ ]:
filter_mf = create_filter(channels, method='matched_filter')
y, transmitted_symbols = simulate_signals(channels)  

### Create a BQM

In [ ]:
bqm = dimod.generators.mimo.spin_encoded_comp(network, 
                                              modulation = 'BPSK', 
                                              transmitted_symbols=transmitted_symbols, 
                                              F_distribution=('binary','real'), 
                                              SNRb=float('inf'),
                                              F=channels,
                                              y=y)

## Decode Received Signals

TBD: move code to helpers

In [ ]:
import numpy as np
from dwave.system import FixedEmbeddingComposite

sampler = FixedEmbeddingComposite(qpu, embedding)

sampleset = sampler.sample(bqm, num_reads=30, annealing_time=200, label='Notebook - comp')

print(f"Decoded with a success rate of {round(100*sum(np.array(list(sampleset.first.sample.values())) == transmitted_symbols.flatten())/num_tx)}%.")

In [ ]:
v = apply_filter(filter_mf, y) 
success_rate = round(100*sum(v.flatten() == transmitted_symbols.flatten())/num_tx)     

print(f"Decoded with a success rate of {success_rate}%.")